In [4]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


In [5]:
# Load environment variables in a file called .env

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [6]:
openai = OpenAI()

In [7]:
class Website:
# A Utility class to represent a website that we have scraped
    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [8]:
ed = Website("https://www.tomshardware.com/pc-components/cpus/intels-fastest-gaming-chip-hits-all-time-low-for-black-friday-core-i9-14900k-drops-to-usd413")
print(ed.title)
print(ed.text)

Intel's fastest gaming chip hits all-time low for Cyber Monday — Core i9-14900K drops to $413 | Tom's Hardware
Skip to main content
Open menu
Close menu
Tom's Hardware
Tom's Hardware
Search
Search Tom's Hardware
RSS
US Edition
UK
US
Australia
Canada
Best Picks
Raspberry Pi
CPUs
GPUs
3D Printers
News
Coupons
More
Newsletter
Reviews
PC Components
Motherboards
SSDs
PC Building
Monitors
Laptops
Gaming
Cooling
RAM
Power Supplies
Cases
3D Printers
Desktops
Overclocking
Peripherals
About Us
Forums
Trending
Black Friday SSD Deals Live
15 under-$50 gadgets
Black Friday Monitor Deals Live
Best Black Friday Tech Deals
Core Ultra 9 285K
PC Components
CPUs
Intel's fastest gaming chip hits all-time low for Cyber Monday — Core i9-14900K drops to $413
Deals
By
Roshan Ashraf Shaikh
published
30 November 2024
Intel's flagship Raptor Lake Refresh CPU drops to a very attractive price.
Comments
(1)
When you purchase through links on our site, we may earn an affiliate commission.
Here’s how it works
.
(Imag

Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

A system prompt that tells them what task they are performing and what tone they should use

A user prompt -- the conversation starter that they should reply to

In [9]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [10]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [11]:
print(user_prompt_for(ed))

You are looking at a website titled Intel's fastest gaming chip hits all-time low for Cyber Monday — Core i9-14900K drops to $413 | Tom's Hardware
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Skip to main content
Open menu
Close menu
Tom's Hardware
Tom's Hardware
Search
Search Tom's Hardware
RSS
US Edition
UK
US
Australia
Canada
Best Picks
Raspberry Pi
CPUs
GPUs
3D Printers
News
Coupons
More
Newsletter
Reviews
PC Components
Motherboards
SSDs
PC Building
Monitors
Laptops
Gaming
Cooling
RAM
Power Supplies
Cases
3D Printers
Desktops
Overclocking
Peripherals
About Us
Forums
Trending
Black Friday SSD Deals Live
15 under-$50 gadgets
Black Friday Monitor Deals Live
Best Black Friday Tech Deals
Core Ultra 9 285K
PC Components
CPUs
Intel's fastest gaming chip hits all-time low for Cyber Monday — Core i9-14900K drops to $413
Deals
By
Roshan Ashraf Shaikh
published
30 Novemb

Messages

The API from OpenAI expects to receive messages in a particular structure. Many of the other APIs share this structure:

[
    {"role": "system", "content": "system message goes here"},
    
    {"role": "user", "content": "user message goes here"}
]

In [12]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [13]:
messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': "You are looking at a website titled Intel's fastest gaming chip hits all-time low for Cyber Monday — Core i9-14900K drops to $413 | Tom's Hardware\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nSkip to main content\nOpen menu\nClose menu\nTom's Hardware\nTom's Hardware\nSearch\nSearch Tom's Hardware\nRSS\nUS Edition\nUK\nUS\nAustralia\nCanada\nBest Picks\nRaspberry Pi\nCPUs\nGPUs\n3D Printers\nNews\nCoupons\nMore\nNewsletter\nReviews\nPC Components\nMotherboards\nSSDs\nPC Building\nMonitors\nLaptops\nGaming\nCooling\nRAM\nPower Supplies\nCases\n3D Printers\nDesktops\nOverclocking\nPeripherals\nAbout Us\nForums\nTrending\nBlack Friday SSD 

In [15]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [16]:
summarize("https://www.tomshardware.com/pc-components/cpus/intels-fastest-gaming-chip-hits-all-time-low-for-black-friday-core-i9-14900k-drops-to-usd413")

"# Summary of Intel's Fastest Gaming Chip Cyber Monday Deal\n\nThe article from Tom's Hardware discusses the **Intel Core i9-14900K**, which has dropped to an all-time low price of **$413.39** on Amazon as part of Cyber Monday deals. The Core i9-14900K is a high-performance CPU featuring 24 cores and 32 threads, capable of reaching clock speeds of up to **6 GHz**. It is compatible with existing LGA1700 motherboards, making it a viable upgrade option.\n\n### Key Details:\n- **Features**: 24-core (8 P-cores, 16 E-cores), supports on-chip graphics.\n- **Pricing**: Regularly priced at **$619**, now discounted to **$413.39**.\n- **Competitor Product**: The Core i9-14900KF variant is available with a $40 coupon, reducing its price to **$396**.\n- **Performance**: The chip is recognized for gaming, ranking among the best CPUs in its category.\n- **Critical Updates**: Intel has released microcode updates to resolve previous voltage issues affecting stability.\n\n### Conclusion:\nThe article hi

In [17]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [18]:
display_summary("https://www.tomshardware.com/pc-components/cpus/intels-fastest-gaming-chip-hits-all-time-low-for-black-friday-core-i9-14900k-drops-to-usd413")

# Summary of Intel’s Fastest Gaming Chip Sale on Tom's Hardware

Tom's Hardware reports that the Intel Core i9-14900K, part of Intel's 14th Generation Raptor Lake Refresh lineup, has reached a new low price of **$413.39** during the Cyber Monday sales on Amazon. This CPU features **24 cores and 32 threads** with impressive clock speeds of up to **6 GHz** for P-cores and **4.4 GHz** for E-cores. The sale is part of ongoing Black Friday deals and is particularly attractive due to a **$25 discount**.

The Core i9-14900K is appreciated for its on-chip graphics, providing added utility for users without a separate graphics card. There is also a mention of its variant, the **Core i9-14900KF**, which offers a **$40 coupon** discount, bringing its price down to **$396**.

### Key Points:
- **Price Drop**: Intel Core i9-14900K now at $413.39 (down from $619).
- **Specifications**: 24-core, 32-thread with high clock speeds; compatible with LGA1700 motherboards.
- **Variants**: Core i9-14900KF available for $396 with a coupon.
- **Issues Addressed**: Previously reported voltage issues have been remedied with microcode updates from Intel.

This article highlights competitive pricing amidst the Black Friday weekend and focuses on finding good deals in the PC hardware market.

In [19]:
display_summary("https://cnn.com")

# CNN Overview

**CNN** is a leading news platform that provides breaking news, latest updates, and a variety of video content across multiple topics, including but not limited to:

- **US News**
- **World Events**
- **Politics**
- **Business**
- **Health**
- **Entertainment**
- **Sports**

The site features segments on ongoing global conflicts such as the Ukraine-Russia War and the Israel-Hamas War. Current headlines include:

- Syrian rebels have taken control of most of Aleppo, challenging the regime of Bashar al-Assad amidst significant international attention.
- A major medical malpractice case resulted in a $412 million payout, marking one of the largest in US history.
- In sports, a brawl occurred during the Michigan-Ohio State football game, which has garnered considerable media coverage.

Additionally, CNN covers trending cultural events, technological developments, and offers various podcasts and videos for diverse audience interests. The site emphasizes user engagement and offers a feedback mechanism for ad relevance and technical issues.

In [20]:
display_summary("https://anthropic.com")

# Summary of Anthropic Website

Anthropic is an AI safety and research company focused on developing reliable and beneficial AI systems. The website highlights their flagship AI model, **Claude 3.5 Sonnet**, which is described as their most intelligent AI to date, now available for use. 

## Key Offerings:
- **Claude AI Models**: Including the latest **Claude 3.5 Sonnet** and **Claude 3.5 Haiku**.
- **API Services**: Designed for businesses to incorporate Claude into their operations for increased efficiency and revenue generation.
- **Enterprise Solutions**: Tailored services for enterprise clients.

## Recent Announcements:
- **October 22, 2024**: Introduced updated models: Claude 3.5 Sonnet and Claude 3.5 Haiku.
- **September 4, 2024**: Launched Claude for Enterprise.
- **Research Highlights**: Previous releases include important discussions on AI safety and research developments.

The company emphasizes its interdisciplinary team, which combines expertise in machine learning, physics, policy, and product design to advance AI technology focused on safety.